In [208]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [209]:
State = namedtuple('State', ['taken', 'not_taken'])

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

In [210]:
PROBLEM_SIZE = 10
NUM_SETS = 15
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

## A*

In [231]:
for i in range(len(SETS)):
    print(SETS[i])


state = State({0,1,2,3}, {3,4,5})
print()
print( covered(state) )
print( sum(covered(state)) )


state = State({1, 2, 3}, {4, 5, 6})
new_state


[False False False False False False False  True False False]
[False False False False False False False  True False False]
[ True  True False  True False False False False False False]
[False False False False False False False False False  True]
[False  True False False  True False  True False False  True]
[False False False  True False False False False False False]
[False  True False False False  True False False False False]
[False False False False False False  True False False False]
[False False  True False  True False  True False False False]
[False  True  True  True False  True False False  True False]
[ True False False False False False  True False False  True]
[ True False False False False False False False False False]
[False False  True False  True False False False False False]
[ True False False  True False False False False  True  True]
[False False False False False False  True False  True False]

[ True  True False  True False False False  True False  True]
5


In [244]:
#evaluate the distance considering the amount of total taken and not_taken in the total need
def h(state):
    #find the set with the max size
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    #vector with the total area covered ( like [T,F,F,T..])
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

#it seems a very dumb approc but that get the better soluction than the other, in fact we count the total number of T in the SETS and the T that i have found
#so i return the num of remains tiles and more tile i have left more possibility i have for the solution
#i put total outside because i need to calculate it only one time
total_tiles = sum(sum(s) for s in SETS)
def h4(state):
    actual_tiles = sum(sum(s) for s in [SETS[i] for i in state.taken])
    optimistic_estimate = total_tiles - actual_tiles
    return optimistic_estimate

#try to give only an extra point if the previous state is better than the previous
def h5(state, old_state):
    if( sum(covered(state)) > sum(covered(old_state)) ):
        return 1
    else:
        return 0
    
def f2(state, old_state):
    return len(state.taken) + h5(state, old_state)

def f(state):
    return len(state.taken) + h4(state)

In [246]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
            #frontier.put((f2(new_state, current_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

10it [00:00, 1794.97it/s]

Solved in 10 steps (10 tiles)
